In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
import numpy as np
from glob import glob

In [5]:
#resize all images
IMAGE_SIZE = [224,224]
train_path = 'train'
test_path = 'test'

In [6]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [7]:
#dont train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [8]:
train_folders = glob('train/*')
train_folders

['train\\without_mask', 'train\\with_mask']

In [9]:
test_folders = glob('test/*')
test_folders

['test\\without_mask', 'test\\with_mask']

In [10]:
#flattern layer
x = Flatten()(vgg16.output)

In [11]:
prediction = Dense(2,activation='softmax')(x)
#create model
model = Model(inputs=vgg16.input,outputs=prediction)

In [12]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3066 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 767 images belonging to 2 classes.


In [18]:
#fit the model
r = model.fit_generator(training_set,
                        validation_data = test_set,
                        epochs=20,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Epoch 1/20


C:\ProgramData\Anaconda3\envs\gpu\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


96/96 [==============================] - 414s 4s/step - loss: 0.2221 - accuracy: 0.9185 - val_loss: 0.1001 - val_accuracy: 0.9687
Epoch 2/20
96/96 [==============================] - 84s 880ms/step - loss: 0.0551 - accuracy: 0.9860 - val_loss: 0.0442 - val_accuracy: 0.9896
Epoch 3/20
96/96 [==============================] - 39s 410ms/step - loss: 0.0455 - accuracy: 0.9863 - val_loss: 0.0292 - val_accuracy: 0.9896
Epoch 4/20
96/96 [==============================] - 36s 379ms/step - loss: 0.0342 - accuracy: 0.9892 - val_loss: 0.1035 - val_accuracy: 0.9648
Epoch 5/20
96/96 [==============================] - 36s 380ms/step - loss: 0.0255 - accuracy: 0.9935 - val_loss: 0.0269 - val_accuracy: 0.9909
Epoch 6/20
96/96 [==============================] - 37s 380ms/step - loss: 0.0181 - accuracy: 0.9961 - val_loss: 0.0139 - val_accuracy: 0.9883
Epoch 7/20
96/96 [==============================] - 37s 380ms/step - loss: 0.0223 - accuracy: 0.9938 - val_loss: 0.0672 - val_accuracy: 0.9700
Epoch 8/20
9

In [19]:
from keras.models import load_model
model.save('Face_Mask_Detector_vgg16.h5')